In [1]:
import os
import sys

# change to upper dir to use all custom libs (won't be needed if run from main scripts)
sys.path.insert(1, os.path.join(sys.path[0], '..'))

In [22]:
import yadisk
import dotenv

dotenv.load_dotenv(dotenv.find_dotenv())
client = yadisk.Client(token=os.environ["YA_DISK_TOKEN"])

In [23]:
# from scripts.helpers.yadisk_manager import download_model

# download_model(
#     client=client,
#     experiment="/bs-diploma/experiments/kaggle/ru-mhr.both_direction.mari-parallel.nllb.upd_tokenizer.finally_fixed",
#     folder='models',
#     checkpoint='final',
# )

In [4]:
from scripts.helpers.model_manager import NLLB200Model

model = NLLB200Model.from_folder(
    model_path='models/model', 
    tokenizer_path='models/tokenizer', 
    convert_to_float16=True, 
    device='auto'
)

C:\Users\cutefluffyfox\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [7]:
mhr_text = model.translate_single('Как твои дела, дорогой друг?', 'ru', 'mhr')
rus_text = model.translate_single(mhr_text, 'mhr', 'ru')
rus_text, mhr_text

('Черт товарищ, а тебе-то что делают?', 'Шӱметный йолташ, тыйын пашажат кузе?')

In [10]:
from scripts.benchmark.flores_dev import benchmark

benchmark(model, 'ru-mhr')
benchmark(model, 'mhr-ru')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 125/125 [05:12<00:00,  2.50s/it]


In [11]:
from scripts.helpers.metric_manager import BLEU, BLEURT, BertScore, CHRF, GoogleBLEU

metrics = [
    BLEU(),
#     BLEURT(),
#     BertScore(),
    CHRF(),
#     GoogleBLEU()
]

Using the latest cached version of the module from C:\Users\cutefluffyfox\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--bleu\9e0985c1200e367cce45605ce0ecb5ede079894e0f24f54613fca08eeb8aff76 (last modified on Tue Nov  5 05:51:14 2024) since it couldn't be found locally at evaluate-metric--bleu, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from C:\Users\cutefluffyfox\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--chrf\d244bab9383988714085a8dacc4871986d9f025398581c33d6b2ee22836b4069 (last modified on Tue Nov  5 05:54:58 2024) since it couldn't be found locally at evaluate-metric--chrf, or remotely on the Hugging Face Hub.


In [17]:
import pandas as pd
from scripts.helpers.path_manager import DataManager



dm = DataManager()
translations = [
    dm.get_path('benchmark', 'ru-mhr', 'mhr_both_dir_simple.ru_mhr.txt'),
    dm.get_path('benchmark', 'mhr-ru', 'mhr_both_dir_simple.mhr_ru.txt'),
    
    dm.get_path('benchmark', 'mhr-ru', 'madlad400.mhr_ru.txt'),
    dm.get_path('benchmark', 'ru-mhr', 'madlad400.ru_mhr.txt'),
#     dm.get_path('benchmark', 'tat-ru', 'madlad400.tat_mhrru.txt'),
#     dm.get_path('benchmark', 'ru-tat', 'madlad400.ru_tat.txt'),
#     dm.get_path('benchmark', 'kaz-ru', 'madlad400.kaz_ru.txt'),
#     dm.get_path('benchmark', 'ru-kaz', 'madlad400.ru_kaz.txt'),
    
    dm.get_path('benchmark', 'mhr-ru', 'nllb200.mhr_ru.txt'),
#     dm.get_path('benchmark', 'tat-ru', 'nllb200.tat_ru.txt'),
#     dm.get_path('benchmark', 'ru-tat', 'nllb200.ru_tat.txt'),
#     dm.get_path('benchmark', 'kaz-ru', 'nllb200.kaz_ru.txt'),
#     dm.get_path('benchmark', 'ru-kaz', 'nllb200.ru_kaz.txt'),
]

for trans in translations:
    print('==========', trans, '==========')
    src_lang, dst_lang = trans.split('.')[-2].split('_')
    for metric in metrics:
        with open(trans, 'r', encoding='UTF-8') as file:
            res = [eval(line.strip()) for line in file.readlines()]
        df = pd.DataFrame(res)
        res_trans, res_ref = df['translation'].to_list(), df[dst_lang].to_list()

        print(metric.metric_name, ':', metric(
            sources=None,
            targets=res_ref,
            translations=res_trans,
            lang=dst_lang
        ))

========== C:\Users\cutefluffyfox\PycharmProjects\MT-vocab-tuning\data\benchmark\ru-mhr\mhr_both_dir_simple.ru_mhr.txt ==========
bleu : 0.07319940181732666
chrf : 38.2987734504991
========== C:\Users\cutefluffyfox\PycharmProjects\MT-vocab-tuning\data\benchmark\mhr-ru\mhr_both_dir_simple.mhr_ru.txt ==========
bleu : 0.08968501201433417
chrf : 36.77403867823899
========== C:\Users\cutefluffyfox\PycharmProjects\MT-vocab-tuning\data\benchmark\mhr-ru\madlad400.mhr_ru.txt ==========
bleu : 0.14416953753714376
chrf : 38.20947588930588
========== C:\Users\cutefluffyfox\PycharmProjects\MT-vocab-tuning\data\benchmark\ru-mhr\madlad400.ru_mhr.txt ==========
bleu : 0.021880696091946265
chrf : 19.71051192504408
========== C:\Users\cutefluffyfox\PycharmProjects\MT-vocab-tuning\data\benchmark\mhr-ru\nllb200.mhr_ru.txt ==========
bleu : 0.03707498445596439
chrf : 27.610867511783933
